# 4. 좋은 훈련 세트 만들기: 데이터 전처리

- 데이터셋이서 누락된 값을 제거하거나 대체하기
- 머신 러닝 알고리즘을 위해 범주형 데이터 변환하기
- 모델과 관련이 높은 특성 선택하기

## 4.1 누락된 데이터 다루기

일반적으로 누락된 값은 데이터 테이블에 빈 공간이나 예약된 문자열로 채워진다  
숫자가 아니라는 의미(not a number)NaN이나 NULL(관계형 데이터베이스에서 모르는 값을 지칭하는데 주로 사용)과 같은 값을 사용한다  
안타깝게도 대부분의 수치 계산 라이브러리는 누락된 값을 다룰 수 없거나 단순히 이를 무시했을 때 예상치 못한 결과를 만든다  

### 4.1.1 테이블 형태 데이터에서 누락된 값 식별

In [1]:
import pandas as pd
from io import StringIO

csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


StringIO를 이용하면 하드 디스크에 있는 일반 CSV파일 처럼 data_csv에 저장된 문자열을 읽어 pandas DataFrame으로 변환할 수 있다  
아주 큰 DataFrame일 경우 수동으로 누락된 값을 찾는 것은 매우 번거롭다  
isnull method는 cell이 수치 값을 담고 있는지(False)또는 누락되어 있는지(True)를 나타내는 boolean 값이 채워진 DataFrame을 반환한다  

In [2]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

이런 식으로 열마다 누락된 값의 개수를 알 수 있다  

사이킷런의 추정기에 주입하기 전에 values 속성을 사용하여 언제나 numpy array를 얻을 수 있다

In [3]:
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

### 4.1.2 누락된 값이 있는 샘플이나 특성 제외

누락된 데이터를 다루는 가장 쉬운 방법 중 하나는 해당 샘플이나 특성을 완전히 삭제하는 것이다  

In [4]:
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


axis 매개변수를 1로 지정해서 NaN이 하나라도 있는 열을 삭제할 수 있다

In [5]:
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


dropa method는 몇 가지 편리한 매개변수를 제공한다

In [6]:
# 모든 열이 NaN일 때만 행을 삭제한다
# (여기서는 모든 값이 NaN인 행이 없기 때문에 전체 배열이 반환된다)
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [7]:
#실수 값이 4개보다 작은 행을 삭제한다
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [8]:
#특정 열에 NaN이 있는 행만 삭제한다
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


너무 많은 데이터를 제거하면 안정된 분석이 불가능할 수 있다  
너무 많은 특성 열을 제거하면 분류기가 클래스를 구분하는 데 필요한 중요한 정보를 잃을 위험이 있다  

### 4.1.3 누락된 값 대체

보간 기법을 사용하여 데이터셋에 있는 다른 훈련샘플로부터 누락된 값을 추정할 수 있다  
가장 흔한 보간 기법 중 하나는 평균으로 대체하는 것이다  
각 특성 열의 전체 평균으로 누락된 값을 바꾸는 것이다  

In [9]:
from sklearn.preprocessing import Imputer
imr = Imputer(missing_values='NaN', strategy='mean', axis=0)
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

C:\Users\ukida\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

axis = 0을 axis = 1로 바꾸면 행의 평균이 계산된다  
strategy 매개변수에 설정할 수 있는 다른 값은 median(데이터를 순서대로 나열했을 떄 중간에 위치한 값), most_frequent(가장 많이 나타난 값)으로 누락된 값을 대체한다

### 4.1.4 사이킷런 추정기 API 익히기

Imputer 클래스는 데이터 변환에 사용되는 사이킷런의 transform 클래스다  
이런 추정기의 주요 method 두개는 fit과 transform이다  
fit method를  사용하여 훈련 데이터에서 모델 parmeter을 학습한다  
transform method를 사용하여 학습한 parmeter로 데이터를 변환한다  
변환하려는 데이터의 배열은 모델 학습에 사용된 데이터의 특성 개수와 같아야한다  

<img src='https://thebook.io/img/007022/p141.jpg'>

<img src='https://thebook.io/img/007022/p142.jpg'>

## 4.2 범주형 데이터 다루기

실제 데이터셋은 하나 이상의 범주형 특성이 포함된 경우가 많다  

### 4.2.1 순서가 있는 특성과 순서가 없는 특성

순서가 있는 특성은 정렬하거나 차례대로 놓을 수 있으므로 범주형 특성이다. ex)티셔츠 사이즈  
순서가 없는 특성은 차례를 부여할 수 없다. ex)티셔츠 컬러  

$\boldsymbol{ 예제 데이터셋 만들기}$

In [10]:
import pandas as pd
df = pd.DataFrame([
                  ['green', 'M', 10.1, 'class1'],
                  ['red', 'L', 13.5, 'class2'], 
                  ['blue', 'XL', 15.3, 'class1']])
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


### 4.2.2 순서 특성 매핑

학습 알고리즘이 순서 특성을 올바르게 인식하려면 범주형의 문자열 값을 정수로 바꾸어야 한다  
안타깝지만 size 특성의 순서를 올바르게 자동으로 바꾸어 주는 함수는 없기 때문에 매핑 함수를 직접 만들어야 한다  
여기서는 특성 간의 산술적인 차이를 이미 알고 있다 가정한다  
ex)
$$
XL=L + 1 = M + 2
$$

In [11]:
size_mapping = {
                'XL': 3,
                'L': 2,
                'M': 1}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


만약 나중에 정수 값을 다시 원래 문자열 표현으로 바꾸고 싶다면 간단히 거꾸로 매핑하는 딕셔너리 inv_size_mapping={v: k for k, v in size_mapping.items()}를 정의하면 된다  
size_mapping 딕셔너리와 비슷하게 pandas의 map method를 사용하여 변환된 특성 열에 적용할 수 있다

In [12]:
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

### 4.2.3 클래스 레이블 인코딩

사이킷런의 분류 추정기 대부분은 자체적으로 클래스 레이블을 정수로 변환해 주지만 사소한 실수를 방지하기 위해 클래스 레이블을 정수 배열로 전달하는 것이 좋다  
특정 문자열 레이블에 할당한 정수는 아무런 의미가 없다  
emumerate를 사용하여 클래스 레이블을 0부터 할당한다

In [13]:
import numpy as np
class_mapping = {label: idx for idx, label in
                 enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

그 다음 매핑 딕셔너리를 사용하여 class label을 정수로 변환한다

In [14]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


매핑 딕셔너리의 키-값 쌍을 뒤집어서 변환된 class label을 다시 원본 문자열로 바꿀 수 있다  

In [15]:
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


다른 방법으로 사이킷런에 구현된 LabelEncoder class를 사용하면 편리하다

In [16]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0])

inverse_transform method를 사용하면 정수 클래스 레이블을 원본 문자열 형태로 되돌릴 수 있다  

In [17]:
class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

### 4.2.4 순서가 없는 특성에 원-핫 인코딩 적용

In [18]:
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

Numpy 배열X의 첫번 째 열은 다음과 같은 color 값을 갖는다

- blue = 0  
- green = 1  
- red = 2  



컬러 값에는 순서가 없지만 학습 알고리즘은 순서가 있다고 가정할 것이다  
이 가정이 옳지 않지만 알고리즘이 의미 있는 결과를 만들 수 있다  
하지만 이 결과가 최선은 아닐 것이다  
이 문제를 해결하기 위한 통상적인 방법은 one-hot encoding 기법이다  
이방식의 아이디어는 순서 없는 특성에 들어 있는 고유한 값마다 새로운 더미(dummy)특성을 만드는 것이다  
여기서 color의 특성을 세개의 새로운 특성인 blue, green, red로 변환한다  
이진 값을 사용하여 특정 샘플의 color를 나타낸다  
예를 들어 blue샘플은 blue = 1. green = 0, red = 0으로 인코딩 된다  

In [19]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(categorical_features=[0])
ohe.fit_transform(X).toarray()

C:\Users\ukida\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\ukida\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


array([[ 0. ,  1. ,  0. ,  1. , 10.1],
       [ 0. ,  0. ,  1. ,  2. , 13.5],
       [ 1. ,  0. ,  0. ,  3. , 15.3]])

원-핫 인코딩으로 더미 변수를 만드는 더 편리한 방법은 판다스의 get_dummies 메서드를 사용하는 것이다  

In [20]:
pd.get_dummies(df[['price', 'color', 'size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


원-핫 인코딩된 데이터셋을 사용할 때 다중 공선성문제를 유념해야한다  
특성 간의 상관관계가 높으면 역행렬을 계산하기 어려워 수치적으로 불안정해진다  
변수 간의 상관관계를 감소하려면 원-핫 인코딩된 배열에서 특성 열 하나를 삭제한다  
이렇게 특성을 삭제해도 잃는 정보는 없다  

get_dummies를 사용할 때 drop_first 매개변수를 True로 지정하여 첫번째 열을 삭제할 수 있다

In [21]:
pd.get_dummies(df[['price', 'color', 'size']],
              drop_first=True)

,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0


OneHotEncoder는 열을 삭제하는 매개변수가 없다  
그 대신 다음 코드처럼 간단하게 원-핫 인코딩된 넘파이 배열을 슬라이싱할 수 있다  

In [22]:
ohe = OneHotEncoder(categorical_features=[0])
ohe.fit_transform(X).toarray()[:, 1:]

C:\Users\ukida\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\ukida\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


array([[ 1. ,  0. ,  1. , 10.1],
       [ 0. ,  1. ,  2. , 13.5],
       [ 0. ,  0. ,  3. , 15.3]])

## 4.3 데이터셋을 훈련 세트와 테스트 세트로 나누기

모델을 실전에 투입하기 전에 테스트 세트에 있는 레이블과 예측을 비교합니다  
이는 편향되지 않은 성능을 측정하기 위해서이다  

In [25]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/'
                     'ml/machine-learning-databases/'
                     'wine/wine.data', header=None)
df_wine.columns = ['Class label', 'Alcohol',
                   'Malic acid', 'Ash',
                   'Alcalinity of Ash', 'Magnesium',
                   'Total phenols', 'Flavanoids',
                   'Nonflavanoid phenols',
                   'Proanthocyanins',
                   'Color intensity', 'Hue',
                   'OD280/OD315 of diluted wines',
                   'Proline']
print('클래스 레이블', np.unique(df_wine['Class label']))

클래스 레이블 [1 2 3]


In [26]:
df_wine.head()

,Class label,Alcohol,Malic acid,Ash,Alcalinity of Ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


사이킷런의 model_selection 모듈에 있는 train_test_split 함수를 사용하면 가장 간편하게 데이터셋을 랜덤한 훈련 세트와 테스트 세트로 나눌 수 있다  

In [27]:
from sklearn.model_selection import train_test_split
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test =\
    train_test_split(X, y,
                     test_size = 0.3,
                     random_state = 0,
                     stratify = y)

인덱스 1에서 13까지의 특성을 넘파이 배열로 변환하여 변수 X에 할당한다  
첫 번째 열릐 클래스 레이블은 변수 y에 할당한다  
train_test_split 함수를 사용하여 X와 y를 랜덤하게 훈련 세트와 테스트 세트로 분할한다  
와인 샘플의 30%가 테스트 세트에 할당된다  
stratify 매개변수를 사용하여 훈련 세트와 테스트 세트에 있는 클래스 비율이 원본 데이터셋과 동일하게 유지된다

## 4.4 특성 스케일 맞추기

스케일이 다른 특성을 맞추는 대표적인 방법 두가지에는 정규화와 표준화가 있다  
재부분의 정규화는 특성의 스케일을 [0,1] 범위에 맞투는 것이다  
최소-최대 스케일 변환의 특별한 경우이다  
데이터를 정규화하기 위해 각 특성의 열마다 최소-최대 스케일 변환을 적용하여 샘플$x^{\left(i\right)}$에서 새로운 값 $x_{norm}^{\left(i\right)}$을 계산한다  

$$
x_{norm}^{i} = \frac{x^{\left(i\right)} - x_{min}}{x_{max} - x_{min}}
$$

In [28]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.fit_transform(X_test)

최소-최대 스케일 변환을 통한 정규화는 범위가 정해진 값이 필요할 때 유용하게 사용할 수 있는 일반적인 기법이다  
표준화는 많은 머신 러닝 알고리즘, 특히 경사 하강법 같은 최적화 알고리즘에서 널리 사용된다  

로지스틱 회귀와 SVM 같은 여러 선형 모델은 가중치를 0 또는 0에 가까운 작은 난수로 초기화한다  
표준화를 사용하면 특성의 평균을 0에 맞추고 표준편차를 1로 만들어 정규 분포와 같은 특징을 가지도록 만든다  
이는 가중치를 더 쉽게 학습할 수 있도록 만든다  
표준화는 이상치 정보가 유지되기 때문에 제한된 범위로 데이터를 조정하는 최소-최대 스케일 변환에 비해 알고리즘이 이상치에 덜 민감하다  
표준화 공식은 다음과 같다

$$
x_{std}^{\left(i \right)} = \frac{x^{\left(i \right)} - \mu_{x}}{\sigma_{x}}
$$

여기서 $\mu_{x}$는 어떤 특성의 샘플 평균이고 $\sigma_{x}$는 그에 해당하는 표준편차이다  

- 0에서 5까지 숫자로 이루어진 간단한 예시 데이터셋에 표준화와 정규화 두 개의 트성 스케일 변환 기법을 적용한 결과
<img src='https://blog.kakaocdn.net/dn/blnW1w/btqBJ8IjTH6/TfKwXnkRvviJ2cSwscAUpK/img.png'>

In [29]:
ex = np.array([0, 1, 2, 3, 4, 5])
print('표준화:', (ex - ex.mean()) / ex.std())

표준화: [-1.46385011 -0.87831007 -0.29277002  0.29277002  0.87831007  1.46385011]


In [30]:
print('정규화', (ex - ex.min()) / (ex.max() - ex.min()))

정규화 [0.  0.2 0.4 0.6 0.8 1. ]


MinMaxScaler 클래스와 비슷하게 sklearn은 표준화를 위한 클래스도 제공한다

In [31]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

여기서 학습한 파라미터로 테스트 세트와 새로운 데이터 포인트를 모두 변환한다  

## 4.5 유용한 특성 선택

모델이 테스트 세트보다 훈련 세트에서 성능이 높다면 과대적합일 확률이 높다  
새로운 데이터에는 잘 일반화하지 못하기 때문에 모델 분산이 크다  
과대적합의 이유는 주어진 훈련데이터에 비해 모델이 너무 복잡하기 때문이다
일반화 오차를 감소시키기 위해 많이 사용하는 방법은 다음과 같다  

- 더 많은 훈련 데이터를 모은다  
- 규제를 통해 복잡도를 제한한다  
- 파라미터 개수가 적은 간단한 모델을 선택한다  
- 데이터 차원을 줄인다  

### 4.5.1 모델 복잡도 제한을 위한 L1 규제와 L2규제

L2 규제는 개별 가중치 값을 제한하여 모델 복잡도를 줄이는 한 방법이다  
가중치 벡터 $\mathbf{w}$의 L2 규제는 다음과 같이 정의한다
$$
L2:\left \| \mathbf{w}\right \|_{2}^{2} = \sum_{j=1}^{m}w_{j}^{2}
$$

모델 복잡도를 줄이는 또 다른 방법은 L1 이다  
$$
L1:\left \| \mathbf{w} \right \|_{1} = \sum_{j=1}^{m}\left|w_{j}\right|
$$